In [10]:
import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots
import seaborn as sns
import plotly.io as pio
import pandas as pd
import numpy as np
import json

from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

import scipy.optimize 
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import classification_report

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w

# Import Data

In [13]:
# measured data 
a00, a01 = h.import_desired_data("A", "15T")
temp0 =h.normalize(a01["Temp C"]) 
window_sched = a01["Window Open"]


In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a01["DateTime"], y=a01["Window Open"]))
fig.add_trace(go.Scatter(x=a01["DateTime"], y=h.normalize(a01["Temp C"])))

In [12]:
model = sm.tsa.MarkovAutoregression(h.normalize(a01["Temp C"]), k_regimes=2, order=4, switching_ar=False)

result = model.fit()

/Users/julietnwagwuume-ezeoke/_UILCode/windows/localwindowsenv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning:

invalid value encountered in reduce



In [14]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                         Markov Switching Model Results                         
================================================================================
Dep. Variable:                   Temp C   No. Observations:                  381
Model:             MarkovAutoregression   Log Likelihood                1196.722
Date:                  Wed, 14 Jun 2023   AIC                          -2375.445
Time:                          12:49:58   BIC                          -2339.960
Sample:                               0   HQIC                         -2361.366
                                  - 381                                         
Covariance Type:                 approx                                         
                             Regime 0 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6435      0.076      8.478      0.000       0.495       0.792
                             Regime 1 parameters                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6435      0.076      8.478      0.000       0.495       0.792
                           Non-switching parameters                           
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2         0.0001   7.93e-06     13.802      0.000    9.39e-05       0.000
ar.L1          1.8187      0.048     37.515      0.000       1.724       1.914
ar.L2         -0.9246      0.101     -9.164      0.000      -1.122      -0.727
ar.L3          0.2539      0.101      2.516      0.012       0.056       0.452
ar.L4         -0.1551      0.049     -3.197      0.001      -0.250      -0.060
                         Regime transition parameters                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p[0->0]        0.5291   1.07e-09   4.93e+08      0.000       0.529       0.529
p[1->0]        0.4739   7.29e-10    6.5e+08      0.000       0.474       0.474
==============================================================================

Warnings:
[1] Covariance matrix calculated using numerical (complex-step) differentiation.
[2] Covariance matrix is singular or near-singular, with condition number 2.65e+23. Standard errors may be unstable.
"""

In [28]:
markov_guess = h.normalize(result.filtered_marginal_probabilities[0])
markov_zscores = h.calc_zscore(markov_guess)

In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a01["DateTime"], y=a01["Window Open"]))
fig.add_trace(go.Scatter(x=a01["DateTime"], y=markov_guess))
fig.add_trace(go.Scatter(x=a01["DateTime"], y=markov_zscores))

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=a01["DateTime"], y=a01["Window Open"]))
fig.add_trace(go.Scatter(x=a01["DateTime"], y=h.normalize(result.smoothed_marginal_probabilities[0])))

# Support Vector Machine 


In [59]:
def split(train_size):
    test_size = 1 - train_size
    return (train_size, test_size) 

In [65]:
# predictions on windows data 
X = temp0.to_numpy()
y = window_sched.to_numpy()
a, b = split(0.2)
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=a, test_size=b, shuffle=False)

In [66]:
fig = make_subplots(rows=1, cols=2, shared_xaxes=True)
fig.add_trace(go.Scatter(
    y=x_train, 
    mode='markers',
), row = 1, col = 1)

fig.add_trace(go.Scatter(
    y=y_train, 
    mode='lines',
), row = 1, col = 1)

fig.add_trace(go.Scatter(
    y=x_test, 
    mode='markers',
), row = 1, col = 2)

fig.add_trace(go.Scatter(
    y=y_test, 
    mode='lines',
), row = 1, col = 2)

In [67]:
clf = OneClassSVM().fit(x_train.reshape(-1,1))


In [68]:
prediction = clf.predict(x_test.reshape(-1,1))
# ability to check performance 
prediction = np.array([0 if i==-1 else 1 for i in prediction])
prediction 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [69]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

fig.add_trace(go.Scatter(
    y=x_test, 
    mode='markers',
    name="observed temp - predictor variable"
), row = 1, col = 1)

fig.add_trace(go.Scatter(
    y=y_test, 
    mode='lines',
    name="window_sched",
), row = 1, col = 1)


fig.add_trace(go.Scatter(
    y=prediction, 
    mode='markers',
    name="prediction",
), row = 1, col = 1)


In [70]:
print(classification_report(y_test, prediction))


              precision    recall  f1-score   support

         0.0       0.41      0.68      0.52       117
         1.0       0.68      0.41      0.51       191

    accuracy                           0.51       308
   macro avg       0.55      0.55      0.51       308
weighted avg       0.58      0.51      0.51       308

